# Video network recommendation

In [1]:
import networkx as nx
import numpy as np
import json, random
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

## Data preparaition

In [2]:
from read_user2vid import *

In [3]:
graph, users, videos, = read_data("../dataset/user2video.json")
dataset, graph_train = genarate_dataset(graph, users, videos)

 22%|██▏       | 106/492 [00:00<00:00, 1053.05it/s]

BiGraph with (492,1344) nodes and 4559 edges
generating negative entries


100%|██████████| 492/492 [00:00<00:00, 715.55it/s] 


6612 negative entries
generating positive entries
227 positive entries
Dataset info:
0    6612
1     227
Name: link, dtype: int64


In [4]:
dataset.head()

,users,videos,link
0,218,2109,1
1,189,870,1
2,241,16,1
3,95,400,1
4,1295,458,1


In [5]:
# dataset = process_parameters_naive(dataset, graph_train)
dataset = process_parameters_node2vec(dataset, graph_train)

Generating walks (CPU: 1):   0%|          | 0/25 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 25/25 [00:27<00:00,  1.11s/it]


In [6]:
dataset.head()

,users,videos,link,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,...,dim_15,dim_16,dim_17,dim_18,dim_19,dim_20,dim_21,dim_22,dim_23,dim_24
0,218,2109,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,189,870,1,1.490830,3.090773,1.761602,1.617907,3.593301,0.877112,-0.228250,...,-3.814042,-5.449229,2.354221,-1.251389,-3.179302,0.770900,0.534840,2.268995,-3.872360,0.469053
2,241,16,1,2.551647,1.481932,0.336205,3.334350,-2.309990,1.990333,-1.060435,...,1.295722,-2.160716,1.618371,-1.978212,1.337865,2.554049,0.151447,0.395403,-0.535100,0.204718
3,95,400,1,2.260036,1.051847,-0.548619,-2.964388,1.266555,-1.367335,-0.415619,...,-0.829449,1.172786,0.316960,-0.292315,-1.318464,3.856657,2.163518,0.796537,-1.057291,3.056418
4,1295,458,1,-2.894198,-2.595119,2.252299,2.504868,-2.194469,2.496771,0.426829,...,-2.282462,1.236756,4.095197,0.526013,-7.582155,-0.843615,1.212296,-0.410348,-0.129791,-0.684353


## Train model

In [7]:
from sklearn.model_selection import train_test_split
# simple models
from sklearn.linear_model import LogisticRegression # 5zhong moxing
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

In [8]:
predictors = np.array(dataset.iloc[:, 3:])
response = dataset['link']
xtrain, xtest, ytrain, ytest = train_test_split(predictors, response,
                                                test_size = 0.3, 
                                                random_state = 114514)

In [9]:
judge = lambda x, y: 0 if x > y else 1

### logistic classification

In [10]:
# logistic regression
lr = LogisticRegression(class_weight="balanced") # lr = 
lr.fit(xtrain, ytrain)
predictions = lr.predict_proba(xtest) # yong xtest suan chu lai de y
print(roc_auc_score(ytest, predictions[:,1]))
ypred = [judge(predictions[i,0], predictions[i,1]) for i in range(predictions.shape[0])]
print(accuracy_score(ytest, ypred))
print(confusion_matrix(ytest, ypred))

0.9224818877931902
0.9941520467836257
[[1980    1]
 [  11   60]]


### SVC

In [11]:
#input shi xtrain he ytrain
clf = SVC(gamma='auto')
clf.fit(xtrain, ytrain)
predictions = clf.predict(xtest)
#print(predictions)
print(roc_auc_score(ytest, predictions))
print(accuracy_score(ytest, predictions))
print(confusion_matrix(ytest, predictions))

0.922282813488706
0.9941520467836257
[[1980    1]
 [  11   60]]


### DecisionTree

In [12]:
tre = DecisionTreeClassifier(criterion='gini')
tre.fit(xtrain, ytrain)
predictions = tre.predict(xtest)
#print(predictions)
print(roc_auc_score(ytest, predictions))
print(accuracy_score(ytest, predictions))
print(confusion_matrix(ytest, predictions))

0.9152405599675794
0.9936647173489279
[[1980    1]
 [  12   59]]


### RandomForest

In [18]:
ranf = RandomForestClassifier(max_depth=10, random_state=0)
ranf.fit(xtrain, ytrain)
predictions = ranf.predict(xtest)
#print(predictions)
print(roc_auc_score(ytest, predictions))
print(accuracy_score(ytest, predictions))
print(confusion_matrix(ytest, predictions))

0.922282813488706
0.9941520467836257
[[1980    1]
 [  11   60]]


### KNeighbors

In [14]:
kn = KNeighborsClassifier(3)
kn.fit(xtrain, ytrain)
predictions = kn.predict(xtest)
#print(predictions)
print(roc_auc_score(ytest, predictions))
print(accuracy_score(ytest, predictions))
print(confusion_matrix(ytest, predictions))

0.9084507042253521
0.9936647173489279
[[1981    0]
 [  13   58]]
